In [38]:

url = ""
with open("../app.env") as file:
    db_source = file.read()
    url = db_source.split("DB_SOURCE=\"")[1][:-1]
    url="mongodb://localhost:27017"
    print(url)

mongodb://localhost:27017


In [39]:
from pymongo import MongoClient
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
import time
client = MongoClient(url)
db = client.nba

In [40]:
from nba_api.stats.static import teams

def lookupTeam(abbreviation):
    original = abbreviation
    if abbreviation == "SEA":
        abbreviation = "OKC"
    if abbreviation == "NOH" or abbreviation == "NOK" or abbreviation == "NOR":
        abbreviation = "NOP"
    if abbreviation == "NJN":
        abbreviation = "BKN"
    ret = [team for team in teams.get_teams() if team['abbreviation'] == abbreviation]
    if len(ret) == 0:
        print(original)
    return ret[0]

In [41]:
from datetime import datetime
lastModifiedDate = datetime.now()
print(lastModifiedDate)

2022-10-20 11:36:58.135503


In [42]:
# get teams
for team in teams.get_teams():
    team_data = {
        "teamID": team['id'],
        'name': team['nickname'],
        'abbreviation': team['abbreviation'],
        "city": team["city"],
        "lastModifiedDate": lastModifiedDate,
        "league": "NBA",
    }
    if team["abbreviation"] == "OKC":
        team_data["alternate_abbreviations"] = ["SEA"]
    if team["abbreviation"] == "NOP":
        team_data["alternate_abbreviations"] = ["NOK", "NOH"]
    if team["abbreviation"] == "BKN":
        team_data["alternate_abbreviations"] = ["NJN"]
    db.teams.update_many({"teamID": team['id']}, {"$set": team_data}, True)

In [43]:
def game_exists(games, game_id):
    # for game in games:
        # if game["gameID"] == game_id:
        #     if "lastModifiedDate" in game.keys():
        #         # last modified after quality assurance date
        #         if game["lastModifiedDate"] > datetime.strptime("2022-04-20T11:50:00", "%Y-%m-%dT%H:%M:%S"):
        #             return True
    return False

In [44]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import boxscoresummaryv2
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscoremiscv2
from nba_api.stats.endpoints import leaguedashptstats

sleeptime = .6

def get_game_data(g, count, playoffs):
    try:
        useDate = g["GAME_DATE"]
        time.sleep(sleeptime)
        boxscorefinder = boxscoretraditionalv2.BoxScoreTraditionalV2(g["GAME_ID"]).get_normalized_dict()
        time.sleep(sleeptime)
        summary = boxscoresummaryv2.BoxScoreSummaryV2(g["GAME_ID"]).get_normalized_dict()["GameSummary"][0]
        time.sleep(sleeptime)
        advanced = boxscoreadvancedv2.BoxScoreAdvancedV2(g["GAME_ID"]).get_normalized_dict()
        time.sleep(sleeptime)
        misc = boxscoremiscv2.BoxScoreMiscV2(g["GAME_ID"]).get_normalized_dict()
        time.sleep(sleeptime)
        players_single_day_passing = leaguedashptstats.LeagueDashPtStats(pt_measure_type="Passing",date_from_nullable=useDate, date_to_nullable=useDate, player_or_team="Player").get_normalized_dict()["LeagueDashPtStats"]
        
        season = summary["SEASON"] + "-"+ str(int(summary["SEASON"][2:])+1)
        #team stats
        # print(count, " ", "VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV")
        for i in range(2):
            game = boxscorefinder["TeamStats"][i]
            if i == 1:
                opponentgame = boxscorefinder["TeamStats"][0]
            else:
                opponentgame = boxscorefinder["TeamStats"][1]
            #if the team's game stats haven't been added already
            if game["PTS"] != None:
                home_or_away = ""
                if game["TEAM_ID"] == summary["HOME_TEAM_ID"]:
                    home_or_away = "home"
                elif game["TEAM_ID"] == summary["VISITOR_TEAM_ID"]:
                    home_or_away = "away"
                else:
                    print("Error: team id not found")
                    exit(0)
                opponent = opponentgame["TEAM_ID"]
                win_or_loss = "loss"
                if game["PTS"] > opponentgame["PTS"]:
                    win_or_loss = "win"
                team_game_data = {
                    "date": summary["GAME_DATE_EST"],
                    "margin": game["PTS"] - opponentgame["PTS"],
                    "points": game["PTS"],
                    "assists": game["AST"],
                    "rebounds": game["REB"],
                    "offensive_rebounds": game["OREB"],
                    "defensive_rebounds": game["DREB"],
                    "turnovers": game["TO"],
                    "steals": game["STL"],
                    "blocks": game["BLK"],
                    "three_pointers_attempted": game["FG3A"],
                    "three_pointers_made": game["FG3M"],
                    "defensive_rating": advanced["TeamStats"][i]["DEF_RATING"],
                    "defensive_rebound_percentage": advanced["TeamStats"][i]["DREB_PCT"],
                    "field_goal_percentage": game["FG_PCT"],
                    "field_goals_attempted": game["FGA"],
                    "field_goals_made": game["FGM"],
                    "gameID":game["GAME_ID"],
                    "home_or_away":home_or_away,
                    "offfensive_rebound_percentage": advanced["TeamStats"][i]["OREB_PCT"],
                    "opponent":opponent,
                    "opponent_assists": opponentgame["AST"],
                    "opponent_effective_field_goal_percentage": opponentgame["FG_PCT"],
                    "opponent_field_goals_attempted": opponentgame["FGA"],
                    "opponent_free_throws_attempted": opponentgame["FTA"],
                    "opponent_points": opponentgame["PTS"],
                    "opponent_rebounds": opponentgame["REB"],
                    "opponent_three_pointers_attempted": opponentgame["FG3A"],
                    "opponent_three_pointers_made": opponentgame["FG3M"],
                    "personal_fouls": misc["sqlTeamsMisc"][i]["PF"],
                    "personal_fouls_drawn": misc["sqlTeamsMisc"][i]["PFD"],
                    "plus_minus_per_hundred": ((int(game["PTS"])-int(opponentgame["PTS"]))/float(advanced["TeamStats"][i]["POSS"]))*100,
                    "possessions": advanced["TeamStats"][i]["POSS"],
                    "season": season,
                    "teamID": game["TEAM_ID"],
                    "win_or_loss":win_or_loss,
                    "playoffs": playoffs,
                    "lastModifiedDate":lastModifiedDate
                }
                # print(db.teamgames.find_one({"teamID": team_game_data["teamID"], "gameID":team_game_data["gameID"]}))
                db.teamgames.replace_one({"teamID": team_game_data["teamID"], "gameID":team_game_data["gameID"]}, team_game_data, True)
                # print("T: ", game["TEAM_NAME"], " vs ", opponentgame["TEAM_NAME"], team_game_data["season"], team_game_data["gameID"])
                # print("\r ",count, game["TEAM_NAME"], " vs ", opponentgame["TEAM_NAME"], team_game_data["season"], team_game_data["gameID"], end="")
        #players stats
        for game in boxscorefinder["PlayerStats"]:
            #if the player played in te game, and the player exists with the provided season (averaged > 10 min in the season), and this player's game stats haven't been added already
            if game["MIN"] != None and db.players.count_documents({"playerID": game["PLAYER_ID"], "seasons":season}) > 0:
                # print(game)
                team1 = lookupTeam(g["MATCHUP"].split(" ")[0])['id']
                team2 = lookupTeam(g["MATCHUP"].split(" ")[2])['id']
                home_or_away = ""
                team = ""
                opponent = ""
                if team1 == game["TEAM_ID"]:
                    team = team1
                    opponent = team2
                    if g["MATCHUP"].split(" ")[1] == "@":
                        #team1 is away
                        home_or_away = "away"
                    else:
                        #team1 is home
                        home_or_away = "home"
                elif team2 == game["TEAM_ID"]:
                    team = team2
                    opponent = team1
                    if g["MATCHUP"].split(" ")[1] == "@":
                        #team1 is away
                        home_or_away = "home"
                    else:
                        #team1 is home
                        home_or_away = "away"
                else:
                    print("ERROR: Team not found")
                margin = 0
                if boxscorefinder["TeamStats"][0]["TEAM_ID"] == team:
                    margin = boxscorefinder["TeamStats"][0]["PTS"] - boxscorefinder["TeamStats"][1]["PTS"]
                else:
                    margin = boxscorefinder["TeamStats"][1]["PTS"] - boxscorefinder["TeamStats"][0]["PTS"]
                win_or_loss="win"
                if margin < 0:
                    win_or_loss="loss"
                potential_assists = 0
                pot_ast_conv = 0
                for passing_player in players_single_day_passing:
                    if passing_player["PLAYER_ID"] == game["PLAYER_ID"]:
                        potential_assists = passing_player["POTENTIAL_AST"]
                if potential_assists > 0:
                    pot_ast_conv = (game["AST"]/potential_assists)*100
                game_data = {
                    "assists":game["AST"], 
                    "potential_assists":potential_assists,
                    "assist_conversion_percentage":pot_ast_conv,
                    "date": g["GAME_DATE"],
                    "margin": margin,
                    "win_or_loss":win_or_loss, #TODO: change this to "outcome"
                    "defensive_rebounds":game["DREB"], 
                    "field_goal_percentage":game["FG_PCT"], 
                    "field_goals_attempted":game["FGA"], 
                    "field_goals_made":game["FGM"], 
                    "free_throws_attempted":game["FTA"],
                    "free_throws_made":game["FTM"], 
                    "free_throws_percentage":game["FT_PCT"],
                    "gameID":game["GAME_ID"],
                    "home_or_away":home_or_away,
                    "minutes": game["MIN"],
                    "offensive_rebounds":game["OREB"], 
                    "opponent":opponent,
                    "personal_fouls":game["PF"], 
                    "playerID":game["PLAYER_ID"],
                    "blocks":game["BLK"],
                    "steals":game["STL"],
                    "points": game["PTS"],
                    "season": season,
                    "team": team,
                    "three_point_percentage":game["FG3_PCT"], 
                    "three_pointers_attempted":game["FG3A"], 
                    "three_pointers_made":game["FG3M"], 
                    "total_rebounds":game["REB"], 
                    "turnovers":game["TO"], 
                    "playoffs": playoffs,
                    "lastModifiedDate":lastModifiedDate
                }
                for advanced_game in advanced["PlayerStats"]:
                    if game_data['playerID'] == advanced_game["PLAYER_ID"]:
                        game_data["assist_percentage"] = advanced_game["AST_PCT"]
                        game_data["defensive_rebound_percentage"] = advanced_game["DREB_PCT"]
                        game_data["effective_field_goal_percentage"] = advanced_game["EFG_PCT"]
                        game_data["offensive_rebound_percentage"] = advanced_game["OREB_PCT"]
                        game_data["true_shooting_percentage"] = advanced_game["TS_PCT"]
                        game_data["usage"] = advanced_game["USG_PCT"]
                for misc_game in misc["sqlPlayersMisc"]:
                    if game_data['playerID'] == misc_game["PLAYER_ID"]:
                        game_data["personal_fouls_drawn"] = misc_game["PFD"]
                res = db.games.replace_one({"playerID": game_data["playerID"], "gameID":game_data["gameID"]}, game_data, True)
    except Exception as e:
        print(e)
        print(g)

In [49]:
#REGULAR SEASON
print(url)
attempt = 0
for attempt in range(5):
    print("attempt: " + str(attempt))
    try:
        teams_found = db.teams.find()
        team_count = 0
        for team in teams_found:
            team_count+=1
            print()
            print(str(team_count), "From team: ", team["name"])
            time.sleep(sleeptime)
            playoffs=False
            gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team["teamID"], league_id_nullable="00", season_type_nullable="Regular Season")
            games = gamefinder.get_normalized_dict()["LeagueGameFinderResults"]
            print("  ", len(games), " total games")
            games = [game for game in games if int(game["SEASON_ID"][1:]) == 2022]
            print("  ", len(games), " recent games")
            found_games = [game for game in db.teamgames.find({"teamID":team["teamID"]})]
            print("  ", len(found_games), " games found in db")
            games = [game for game in games if game_exists(found_games, game["GAME_ID"]) == False]
            print("  ", len(games), " games not in db")

            count=0
            for g in games:
                count+=1
                get_game_data(g, count, playoffs)
        break
    except Exception as e:
        print("failed on attempt: " + str(attempt))
        print(e)
        # print(game)
        time.sleep(sleeptime*10)

mongodb://localhost:27017
attempt: 0

1 From team:  Hawks
   3126  total games
   1  recent games
   180  games found in db
   1  games not in db

2 From team:  Celtics
   3130  total games
   1  recent games
   185  games found in db
   1  games not in db

3 From team:  Cavaliers
   3124  total games
   1  recent games
   157  games found in db
   1  games not in db

4 From team:  Pelicans
   1605  total games
   1  recent games
   163  games found in db
   1  games not in db

5 From team:  Bulls
   3124  total games
   1  recent games
   160  games found in db
   1  games not in db

6 From team:  Mavericks
   3134  total games
   1  recent games
   180  games found in db
   1  games not in db

7 From team:  Nuggets
   3132  total games
   1  recent games
   170  games found in db
   1  games not in db

8 From team:  Warriors
   3124  total games
   1  recent games
   179  games found in db
   1  games not in db

9 From team:  Rockets
   3131  total games
   1  recent games
   155  ga

In [50]:
#PLAYOFFS
attempt = 0
for attempt in range(5):
    print("attempt: " + str(attempt))
    try:
        teams_found = db.teams.find()
        team_count = 0
        for team in teams_found:
            team_count+=1
            print()
            print(str(team_count), "From team: ", team["name"])
            time.sleep(sleeptime)
            gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team["teamID"], league_id_nullable="00", season_type_nullable="PlayIn")
            playins = gamefinder.get_normalized_dict()["LeagueGameFinderResults"]
            gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team["teamID"], league_id_nullable="00", season_type_nullable="Playoffs")
            games = gamefinder.get_normalized_dict()["LeagueGameFinderResults"]
            games = games + playins
            playoffs=True
            print(len(games), " total games")
            games = [game for game in games if int(game["SEASON_ID"][1:]) >= 2022]
            print(len(games), " recent games")
            found_games = [game for game in db.teamgames.find({"teamID":team["teamID"]})]
            print(len(found_games), " games found in db")
            games = [game for game in games if game_exists(found_games, game["GAME_ID"]) == False]
            print(len(games), " games not in db")

            count=0
            for g in games:
                count+=1
                get_game_data(g, count, playoffs)
        break
    except Exception as e:
        print("failed on attempt: " + str(attempt))
        print(e)
        # print(game)
        time.sleep(sleeptime*10)

attempt: 0

1 From team:  Hawks
212  total games
0  recent games
180  games found in db
0  games not in db

2 From team:  Celtics
378  total games
0  recent games
185  games found in db
0  games not in db

3 From team:  Cavaliers
213  total games
0  recent games
157  games found in db
0  games not in db

4 From team:  Pelicans
57  total games
0  recent games
163  games found in db
0  games not in db

5 From team:  Bulls
285  total games
0  recent games
160  games found in db
0  games not in db

6 From team:  Mavericks
227  total games
0  recent games
180  games found in db
0  games not in db

7 From team:  Nuggets
177  total games
0  recent games
170  games found in db
0  games not in db

8 From team:  Warriors
193  total games
0  recent games
179  games found in db
0  games not in db

9 From team:  Rockets
263  total games
0  recent games
155  games found in db
0  games not in db

10 From team:  Clippers
122  total games
0  recent games
175  games found in db
0  games not in db

11 Fr

In [54]:

# if playername (key) (e.g. proposition) exists in db as another name, then use the value
alternate_player_names = {
    "Nicolas Claxton": "Nic Claxton",
    "OG Anunoby": "O.G. Anunoby",
    "Marcus Morris": "Marcus Morris Sr.",
}

In [55]:
# get upcoming game information for players with projections in the database
import requests
json = requests.get("https://cdn.nba.com/static/json/staticData/scheduleLeagueV2.json").json()
dates = json["leagueSchedule"]["gameDates"]
print(len(dates))
lastModifiedDate = datetime.now()
for date in dates:
    for game in [game for game in date["games"] if game["gameDateEst"] > datetime.now().strftime("%Y-%m-%d")]:
        for doc in db.projections.find({"date":game["gameDateEst"][:10], "opponent":game["awayTeam"]["teamTricode"]}):
            # trim whitespace from player name
            playername = doc["playername"].strip()
            if alternate_player_names.get(doc["playername"]):
                playername = alternate_player_names[doc["playername"]]
            player = db.players.find_one({"name":playername}, {"_id":0, "playerID":1})
            if player == None:
                print("!!! Player '"+doc["playername"]+"' not found in players collection!!!")
                continue
            home_player_game = {
                "gameID": game["gameId"],
                "opponent": game["awayTeam"]["teamId"],
                "playoffs": False,
                "date": game["gameDateEst"][:10],
                "lastModifiedDate": lastModifiedDate,
                "season":"2022-23",
                "team": game["homeTeam"]["teamId"],
                "playerID": player["playerID"],
                "win_or_loss":"pending",
            }
            res = db.games.replace_one({"gameID":home_player_game["gameID"], "playerID":home_player_game["playerID"]}, home_player_game, True)
            print('{:20s} {:2s} {:3s} on {:10} {:}'.format(playername, "vs", game["awayTeam"]["teamTricode"], game["gameDateEst"][:10], res.modified_count > 0 and "updated" or "added"))
        for doc in db.projections.find({"date":game["gameDateEst"][:10], "opponent":game["homeTeam"]["teamTricode"]}):
            playername = doc["playername"].strip()
            if alternate_player_names.get(doc["playername"]):
                playername = alternate_player_names[doc["playername"]]
            player = db.players.find_one({"name":playername}, {"_id":0, "playerID":1})
            if player == None:
                print("!!! Player '"+playername+"' not found in players collection!!!")
                continue
            away_player_game = {
                "gameID": game["gameId"],
                "opponent": game["homeTeam"]["teamId"],
                "playoffs": False,
                "date": game["gameDateEst"][:10],
                "lastModifiedDate": lastModifiedDate,
                "season":"2022-23",
                "team": game["awayTeam"]["teamId"],
                "playerID": player["playerID"],
                "win_or_loss":"pending",
            }
            res = db.games.replace_one({"gameID":away_player_game["gameID"], "playerID":away_player_game["playerID"]}, away_player_game, True)
            print('{:20s} {:2s} {:3s} on {:10} {:}'.format(playername, "@", game["homeTeam"]["teamTricode"], game["gameDateEst"][:10], res.modified_count > 0 and "updated" or "added"))

179
James Harden         vs MIL on 2022-10-20 updated
Joel Embiid          vs MIL on 2022-10-20 updated
P.J. Tucker          vs MIL on 2022-10-20 updated
Tobias Harris        vs MIL on 2022-10-20 updated
Tyrese Maxey         vs MIL on 2022-10-20 updated
Brook Lopez          @  PHI on 2022-10-20 updated
Giannis Antetokounmpo @  PHI on 2022-10-20 updated
Grayson Allen        @  PHI on 2022-10-20 updated
Jrue Holiday         @  PHI on 2022-10-20 updated
Anthony Davis        vs LAC on 2022-10-20 updated
LeBron James         vs LAC on 2022-10-20 updated
Lonnie Walker IV     vs LAC on 2022-10-20 updated
Patrick Beverley     vs LAC on 2022-10-20 updated
Russell Westbrook    vs LAC on 2022-10-20 updated
Ivica Zubac          @  LAL on 2022-10-20 updated
Kawhi Leonard        @  LAL on 2022-10-20 updated
Marcus Morris Sr.    @  LAL on 2022-10-20 added
Paul George          @  LAL on 2022-10-20 updated
Reggie Jackson       @  LAL on 2022-10-20 updated
